In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

In [2]:
# 本例数据来源：https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/data

In [3]:
dat = pd.read_csv('./data/train.csv', dtype={'TripType':'category'
                                             ,'VisitNumber':object
                                             , 'Upc':object
                                             , 'FinelineNumber':object})

In [4]:
dat.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


### 判断空值 isna|isnull、notna

In [5]:
na = dat.isna() # notna和isna结果相反， 另外有isnull和isna是一样的，建议用isna

## 补充说明：numpy里面有一个np.nan，并不是空值的意思，而是 not a number

In [6]:
na.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False


In [7]:
# 每一列中有多少行是空值
print(dat.isna().any())
print("---")
for i in na.columns:
    print('%s的空值占比：%.2f%%' %(i, na[i].sum()/dat.shape[0]*100))

TripType                 False
VisitNumber              False
Weekday                  False
Upc                       True
ScanCount                False
DepartmentDescription     True
FinelineNumber            True
dtype: bool
---
TripType的空值占比：0.00%
VisitNumber的空值占比：0.00%
Weekday的空值占比：0.00%
Upc的空值占比：0.64%
ScanCount的空值占比：0.00%
DepartmentDescription的空值占比：0.21%
FinelineNumber的空值占比：0.64%


In [8]:
# 每一行中有多少是空值：
na['naCount'] = na.sum(axis=1)
na.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,naCount
0,False,False,False,False,False,False,False,0
1,False,False,False,False,False,False,False,0
2,False,False,False,False,False,False,False,0
3,False,False,False,False,False,False,False,0
4,False,False,False,False,False,False,False,0


In [9]:
na.naCount.value_counts()

0    642925
2      2768
3      1361
Name: naCount, dtype: int64

### 查看有空值行对应的y分布

In [10]:
y_col_name = 'TripType'
print(dat[dat.isna().any(axis=1)].shape)
empty_y_dist = dat[dat.isna().any(axis=1)][y_col_name].value_counts() 
empty_y_dist = empty_y_dist.reset_index()
empty_y_dist.columns = [y_col_name, 'EmptyCnt']
empty_y_dist['EmptyCntPct']=np.around(empty_y_dist['EmptyCnt']/empty_y_dist['EmptyCnt'].sum(), 4)
empty_y_dist.head()


(4129, 7)


,TripType,EmptyCnt,EmptyCntPct
0,5,2583,0.6256
1,999,352,0.0853
2,40,193,0.0467
3,39,135,0.0327
4,25,116,0.0281


In [11]:
total_y_dist = dat[y_col_name].value_counts()
total_y_dist = total_y_dist.reset_index()
total_y_dist.columns = [y_col_name, 'TotalCnt']
total_y_dist['TotalCntPct']=np.around(total_y_dist['TotalCnt']/total_y_dist['TotalCnt'].sum(), 4)
total_y_dist.head()

,TripType,TotalCnt,TotalCntPct
0,40,174164,0.2692
1,39,95504,0.1476
2,37,38954,0.0602
3,38,29565,0.0457
4,25,27609,0.0427


In [12]:
total_merge_empty = total_y_dist.merge(empty_y_dist, on='TripType', how='left')
total_merge_empty['NonEmptyCnt'] = total_merge_empty.TotalCnt - total_merge_empty.EmptyCnt
total_merge_empty['NonEmptyCntPct'] = np.around(total_merge_empty['NonEmptyCnt']/total_merge_empty['NonEmptyCnt'].sum(), 4)
total_merge_empty['EmptyPctOfTotal'] = total_merge_empty.EmptyCnt/total_merge_empty.TotalCnt
total_merge_empty.head(10)
# 看每个类型的空值比例（EmptyPctOfTotal）
# 看去除空值后，每个类型的占比变化是否较大（比较TotalCntPct和NonEmptyCntPct)
## 就本从本数据集的空值分布来看，空值占比不高，且去除空值后对整体分布影响较小；


,TripType,TotalCnt,TotalCntPct,EmptyCnt,EmptyCntPct,NonEmptyCnt,NonEmptyCntPct,EmptyPctOfTotal
0,40,174164,0.2692,193,0.0467,173971,0.2706,0.001108
1,39,95504,0.1476,135,0.0327,95369,0.1483,0.001414
2,37,38954,0.0602,68,0.0165,38886,0.0605,0.001746
3,38,29565,0.0457,40,0.0097,29525,0.0459,0.001353
4,25,27609,0.0427,116,0.0281,27493,0.0428,0.004202
5,7,23199,0.0359,31,0.0075,23168,0.0360,0.001336
6,8,22844,0.0353,78,0.0189,22766,0.0354,0.003414
7,36,21990,0.0340,49,0.0119,21941,0.0341,0.002228
8,44,20424,0.0316,63,0.0153,20361,0.0317,0.003085
9,42,19468,0.0301,50,0.0121,19418,0.0302,0.002568


### 空值处理： 删除或者填充

#### 1. 删除

In [13]:
# 1. 删除空值行 dropna
dat2 = dat.dropna(axis=0, how='any')
# axis=0表示删除行，axis=1表示删除列
# how='any'表示只要有一个NA就删除整行(列), 如果都所有列（行）都为NA才删除，设置how='all'
# 如果设置参数thresh=2,表示行(行)中NA的数量大于2，才删除
# 如果设置参数subset=['c1', 'c2'], 表示这些列(行)中有NA，所在的行(列)才会被删除；
print(dat2.shape)

(642925, 7)


In [14]:
dat2.isna().sum().sum()

0

#### 2. 填充

In [15]:
# 2. 填充空值 finnna
values = {'Upc':'EMPTY', 'DepartmentDescription':'No Department', 'FinelineNumber':'EMPTY'}
dat3 = dat.fillna(value=values)
dat3[na.naCount==2].head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
1155,44,496,Friday,EMPTY,1,PHARMACY RX,EMPTY
1216,5,521,Friday,EMPTY,1,PHARMACY RX,EMPTY
1373,5,585,Friday,EMPTY,1,PHARMACY RX,EMPTY
1455,5,619,Friday,EMPTY,1,PHARMACY RX,EMPTY
1456,5,619,Friday,EMPTY,1,PHARMACY RX,EMPTY


In [16]:
dat4 = dat.fillna(method='ffill') # ffill(pad)表示用前面一个值填充, backfill(bfill)用后面最近一个有效值的填充
dat4[(dat4.index>1152)&(dat4.index<1157)]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
1153,44,496,Friday,30067399942,1,PHARMACY OTC,3721
1154,44,496,Friday,88530618861,1,BOYS WEAR,1701
1155,44,496,Friday,88530618861,1,PHARMACY RX,1701
1156,44,496,Friday,6700882840,1,CELEBRATION,787


In [17]:
values = {'Upc':'EMPTY', 'DepartmentDescription':'No Department', 'FinelineNumber':'EMPTY'}
dat5 = dat.fillna(value=values, limit=3) # limit=3表示只填充了前3行的NA
dat5[na.naCount==3].head() # 

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
25,26,8,Friday,EMPTY,1,No Department,EMPTY
548,27,259,Friday,EMPTY,3,No Department,EMPTY
549,27,259,Friday,EMPTY,1,No Department,EMPTY
959,999,409,Friday,NaN,-1,NaN,NaN
1116,39,479,Friday,NaN,1,NaN,NaN
